## Analysis

<l>
    <li> Plot graphs.</li>
    <li> Find best hyperparameters.</li>
<l>

In [1]:
import numpy as np
import pandas as pd

In [18]:
results = {}
results["results_1_layer"] = pd.read_csv("results/results_tuning_1_layer.csv", index_col=0)
results["results_2_3_layers"] = pd.read_csv("results/results_tuning_2_3_layers.csv", index_col=0)
results["results_1_layer_maxlen_500"] = pd.read_csv("results/results_tuning_1_layer_maxlen_500.csv", index_col=0)
results["results_2_3_layers_maxlen_500"] = pd.read_csv("results/results_tuning_2_3_layers_maxlen_500.csv", index_col=0)

In [14]:
results["results_1_layer"]["max_num_tokens"] = None
results["results_2_3_layers"]["max_num_tokens"] = None
results["results_1_layer_maxlen_500"]["max_num_tokens"] = 500
results["results_2_3_layers_maxlen_500"]["max_num_tokens"] = 500

In [19]:
df_results = pd.concat(results.values(), ignore_index=True)

In [17]:
# jupyter nbextension enable --py --sys-prefix qgrid
import qgrid
# only required if you have not enabled the ipywidgets nbextension yet
# jupyter nbextension enable --py --sys-prefix widgetsnbextension
#to show a df simply use the below:
qgrid.show_grid(df_results)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…